# CHBE444: Design I, University of Maryland
### Prof. Ganesh Sriram, gsriram@umd.edu
### E.02 and HW 2 Urea Reaction Path LP

# How to Use ``linprog`` When Objective Function Needs to Be Maximized or Contains a Constant

The SciPy function ``linprog`` is a minimization algorithm, designed to find the minimum of an LP. Additionally, it does not accept an objective function with a constant term. This short tutorial goes over how to hack ``linprog`` for these scenarios. 

For demonstration, we will use the familiar urea LP in $[x_1, x_3]$ axes, with the objective function being the yield of urea: $z = \frac{1}{2}x_3$. Thus, the minimum of $z$ in the feasible space is located at $(1,2)$ with $z_{min} = 1$, and the maximum is located at $(1.5,3)$ with $z_{max} = 1.5$. Refer to the feasible space plot at the end of this notebook.

## Using ``linprog`` to Find Minimum

In [1]:
import scipy.optimize as opt

# Urea reaction path LP with CO constraint

c = [0,1/2]  # objective function

A = [[1,-1/2],[-1/3,1/2],[-1,0]]  # inequality constraint matrix
b = [0,1,-1]  # inequality constraint vector

res = opt.linprog(c,A,b,A_eq=None,b_eq=None,bounds=[0,None],method='highs')

print('Optimal x:',res.x)
print('Optimal z:',res.fun)

Optimal x: [1. 2.]
Optimal z: 1.0


#### Summary
+ In the cell above, the vector $\bf{c}$ was defined as $[ 0 \ \frac{1}{2}]$, i.e., $z = 0x_1 + \frac{1}{2}x_3 = \frac{1}{2}x_3$.
+ The minimum of this function in the feasible space is located at $(1,2)$ with $z_{min} = 1$.
+ Unsurprisingly, ``linprog`` found this minimum because that is what it is designed to do.

## Using ``linprog`` to Find Maximum

If we need to find the maximum of $z$, we should ask ``linprog`` to find the minimum of $-z$, because the minimum of $-z$ is the negative of the maximum of $z$. The code for this is:

In [2]:
c = [0,-1/2]  # objective function is negated

# A and b do not have to be defined again if the previous cell was run and the kernel was not restarted

res = opt.linprog(c,A,b,A_eq=None,b_eq=None,bounds=[0,None],method='highs')

print('Optimal x:',res.x)
print('Optimal z:',-res.fun)  # negative sign on z removed at this step (-res.fun instead of res.fun)

Optimal x: [1.5 3. ]
Optimal z: 1.4999999999999998


#### Summary
+ This time, ``linprog`` found the maximum of the function because we made it do so.
+ ``linprog`` minimizes, so when maximizing, you solve ``min(-z)`` and then negate the objective returned by ``linprog``.

## Finding Minimum When Objective Function Has a Constant

Suppose the objective function had a constant term, say, $z = \frac{1}{2}x_3 + \frac{3}{4}$. ``linprog`` does not accept the constant term. However, since this term $\frac{3}{4}$ is constant and does not vary with $x_1$ or $x_3$, the minimum of $z$ will be the minimum of $\frac{1}{2}x_3$ plus $\frac{3}{4}$. Thus, we can initially define $z$ by leaving out the constant term, and add it at the end, as follows.

In [3]:
c = [0,1/2]  # objective function without the constant

# A and b do not have to be defined again if the previous cell was run and the kernel was not restarted

res = opt.linprog(c,A,b,A_eq=None,b_eq=None,bounds=[0,None],method='highs')

print('Optimal x:',res.x)
print('Optimal z:',res.fun+3/4)  # constant is added at this step

Optimal x: [1. 2.]
Optimal z: 1.75


#### Summary
+ As expected, we got the minimum $z_{min} = 1.75$ at $(1,2)$.
+ ``linprog`` does not accept a constant in the objective function. So when a constant is present, add it to the objective returned by ``linprog``.

## Finding Maximum When Objective Function Has a Constant

Let us combine all the preceding tricks to find the maximum of $z = \frac{1}{2}x_3 + \frac{3}{4}$ in the same feasible space. We can do this by asking ``linprog`` to find its minimum of $-z$ without the constant term. At the end, we can remove the negative sign and add the constant term.

In [4]:
c = [0,-1/2]  # objective function is negated, there is no constant

# A and b do not have to be defined again if the previous cell was run and the kernel was not restarted

res = opt.linprog(c,A,b,A_eq=None,b_eq=None,bounds=[0,None],method='highs')

print('Optimal x:',res.x)
print('Optimal z:',-res.fun+3/4)  # negative sign on z removed and constant added

Optimal x: [1.5 3. ]
Optimal z: 2.25


#### Summary
+ As expected, we got the maximum $z_{max} = 2.25$ at $(1.5,3)$. 
+ Remember that ``linprog`` minimizes, so when maximizing, you solve ``min(-z)`` and then negate the objective returned; when a constant is present, add it after negation.

## Desmos Plot of Feasible Space

In [1]:
import chbe444umd as des

expressions = [{'id':'x1', 'latex':'x_1=x', 'hidden':True},  # define x1 as the 'x' variable, x1 is typed as x_1
               {'id':'x3', 'latex':'x_3=y', 'hidden':True},  # define x3 as the 'y' variable, x3 is typed as x_3
               {'id':'const1', 'latex':'x_1-x_3/2<=0', 'color':'#0000ff', 'fillOpacity':0.2},
               {'id':'const2', 'latex':'-x_1/3+x_3/2<=1', 'color':'#0000ff', 'fillOpacity':0.2},
               {'id':'const3', 'latex':'-x_1<=-1', 'color':'#0000ff', 'fillOpacity':0.2},
               {'id':'const4', 'latex':'-x_1<=0', 'color':'#0000ff', 'fillOpacity':0.2},
               {'id':'const5', 'latex':'-x_3<=0', 'color':'#0000ff', 'fillOpacity':0.2},
               {'id':'obj', 'latex':'x_3/2=z', 'color':'#000000', 'fillOpacity':1}]

sliders =[{'variable':'z', 'min':'0', 'max':'4', 'step':'0.1'}]

points = [{'id':'maxpt', 'latex':'(3/2,3)', 'label':'`(1.5,3)`', 'showLabel':True, 'color':'#000000', 'font':1},
          {'id':'max', 'latex':'(3/2,3)', 'label':'`z_{max} = 1.5`', 'showLabel':True, 'color':'#000000', 'font':1},
          {'id':'minpt', 'latex':'(1,2)', 'label':'`(1,2)`', 'showLabel':True, 'color':'#000000', 'font':1},
          {'id':'min', 'latex':'(1,2)', 'label':'`z_{min} = 1`', 'showLabel':True, 'color':'#000000', 'font':1}]

graph_html = des.desmos_graph(expressions,sliders,points,axlim=[-0.2,4,-0.2,4],size=[1000,600])

with open('urea_graph.html',"w",encoding="utf-8") as f:
    f.write(graph_html.get_value())
 
from IPython.display import IFrame
IFrame('urea_graph.html',width='100%',height=400)